# Sorting Notebook

This notebook will download and sort electrophysiology collected using an Intan headstage, in the .rhd format. 

The data is intracranial mouse recording, from a 16 channel microarray. The paper can be found here: https://doi.org/10.1371/journal.pone.0221510


# Getting Set Up

Open a terminal. Make sure "Sorter" environment is active. 

```
conda deactivate
conda activate sorter
```


In [1]:
# Imports
import os
import json
from pathlib import Path

from sorting_scripts import get_file

import spikeinterface.full as si
import probeinterface as pi
import spikeinterface.sorters as ss
from spikeinterface.sorters import run_sorter
from spikeinterface.curation import apply_curation
from spikeinterface.core import load_sorting_analyzer

/opt/anaconda3/envs/sorter311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Set Patient and Session
patient = "raw_intan"
session = "Session1"

In [ ]:
# Set base paths
codespace = Path.home() / "codespace"
base_folder = codespace / "data"
session_location =  base_folder / patient / session
sorted_data = session_location / f"{patient}-{session}-sorted"
sorter_output_folder = sorted_data / f"{patient}-{session}-sorter_folder" 
analyzer_folder = sorted_data / f"{patient}-{session}-analyzer_folder"
os.chdir(session_location)
intan_file = get_file.get_rhd_file(session_location)

# Load recording into spike interface

In [20]:
# Load Recording, creates recording object in memory
rec = si.read_intan(intan_file, stream_id = "0")
rec

IntanRecordingExtractor: 16 channels - 20.0kHz - 1 segments - 24,000,480 samples 
                         1,200.02s (20.00 minutes) - uint16 dtype - 732.44 MiB
  file_path: /data/raw_intan/Session1/raw/Intan RHD file1.rhd

In [ ]:
# Attach probe to recording object

# from probeinterface.plotting import plot_probe, plot_probegroup
probe_path = codespace / "sorting_script/Custom_Probes/neuronexus-A16x1_2mm_50_177_A16.json"

# Load from JSON
probegroup = pi.read_probeinterface(probe_path)

# Extract the single Probe for SpikeInterface
probe = probegroup.probes[0]

# Attach to recording
rec = rec.set_probe(probe)

# Ensure channel count matches 
n_rec = rec.get_num_channels()
n_probe = probe.get_contact_count()

if n_probe != n_rec:
    raise ValueError(f"Probe contacts ({n_probe}) != recording channels ({n_rec}). "
                     f"Pick the correct probe variant or subset/remap accordingly.")


# Load sorter and analyzer if they exist

# If they do not exist, run the sorter and create analyzer

In [ ]:
try:
    # Load sorting object from sorting directory
    sorting_KS4 = ss.read_sorter_folder(sorter_output_folder)

except Exception as e:
    # Do minimal pre-processing
    rec = si.bandpass_filter(rec)
    rec = si.center(rec)
    rec = si.whiten(rec)
    
    # Run Kilosort, in order to create sorting object as well as sorting folder
    sorting_KS4 = run_sorter(
        sorter_name="kilosort4",
        recording=rec,
        folder=sorter_output_folder,
        remove_existing_folder = True,
        verbose = True
    )

write_binary_recording (no parallelization):   0%|          | 0/1201 [00:00<?, ?it/s]

kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 24000480
kilosort.run_kilosort: N seconds: 1200.024
kilosort.run_kilosort: N batches: 401
kilosort.run_kilosort: Preprocessing filters computed in 0.43s; total 0.43s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     7.10 %
kilosort.run_kilosort: Mem used:      5.80 %     |       3.61 GB
kilosort.run_kilosort: Mem avail:    58.49 / 62.10 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:    1.86 %     |      0.27   /    14.58 GB
kilosort.run_kilosort: Allocated:     0.06 %     |      0.01   /    14.58 GB
kilosort.run_kilosor

kilosort4 run time 43.50s


In [ ]:
# Load Sorting Analyzer 
try:
    sorting_analyzer = si.load_sorting_analyzer(analyzer_folder)
    print("Loaded existing SortingAnalyzer")

# Create Sorting Analyzer
except Exception as e:
    print("No valid analyzer found, creating a new one")
    print(f"Reason: {e}")
    recording = si.read_intan(intan_file, stream_id = "0")
    recording = recording.set_probe(probe, in_place=False)
    recording = si.unsigned_to_signed(recording)
    recording_filtered = si.bandpass_filter(recording)
    

    job_kwargs = dict(n_jobs=-1, progress_bar=True, chunk_duration="1s")
    
    sorting_analyzer = si.create_sorting_analyzer(
        sorting=sorting_KS4,
        recording=recording_filtered,
        folder=analyzer_folder,
        overwrite=True,
        format="binary_folder",
        **job_kwargs
    )

    sorting_analyzer.compute({
    "random_spikes": dict(method="uniform", max_spikes_per_unit=500),
    "waveforms": job_kwargs,
    "templates": job_kwargs,
    "noise_levels": {},
    "unit_locations": dict(method="monopolar_triangulation"),
    "isi_histograms": {},
    "correlograms": dict(window_ms=100, bin_ms=5),
    "principal_components": dict(
        n_components=3,
        mode="by_channel_global",
        whiten=True,
        **job_kwargs
    ),
    "quality_metrics": dict(metric_names=["snr", "firing_rate"]),
    "spike_amplitudes": job_kwargs,
    })

estimate_sparsity (workers: 16 processes):   0%|          | 0/1201 [00:00<?, ?it/s]

/home/marco/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface/core/basesorting.py:380: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")


compute_waveforms (workers: 16 processes):   0%|          | 0/1201 [00:00<?, ?it/s]

noise_level (no parallelization):   0%|          | 0/20 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/28 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/28 [00:00<?, ?it/s]

spike_amplitudes (workers: 16 processes):   0%|          | 0/1201 [00:00<?, ?it/s]

In [ ]:
# Manual Sorting
# remove some units
remove_unit_ids = [1, 2]
sorting_analyzer2 = analyzer.remove_units(remove_unit_ids=remove_unit_ids)

# merge units 4 and 5, and separately merge units 7, 8 and 12
merge_unit_groups = [[4, 5], [7, 8, 12]]
sorting_analyzer3 = sorting_analyzer2.merge_units(
    merge_unit_groups=merge_unit_groups,
    censored_period_ms=0.5,
    merging_mode="soft"
)

In [ ]:
from spikeinterface.curation import auto_merge_units

# Or the fully automatic version
template_diff_thresh = [0.05, 0.15, 0.25]
presets = ["x_contaminations"] * len(template_diff_thresh)
steps_params = [
    {"template_similarity": {"template_diff_thresh": i}}
    for i in template_diff_thresh
]

analyzer_merged = auto_merge_units(
    sorting_analyzer,
    presets=presets,
    steps_params=steps_params,
    recursive=True,
    **job_kwargs,
)

#### Run the curation gui from the terminal, Make sure to replace with the correct path

```
sigui --mode=web --curation "/path/to/your/data/raw_intan/Session1/sorted/analyzer_folder"
```






In [ ]:
# Once curation is complete, save as a new sorting analyzer object, and generate the directory. 

# --- inputs ---
curation_filepath = Path(analyzer_folder) / "spikeinterface_gui" / "curation_data.json"
base_out = Path(sorted_data) / f"{patient}-{session}-curated_analyzer.zarr"

# --- sanity checks ---
if not curation_filepath.exists():
    raise FileNotFoundError(
        f"Missing curation file: {curation_filepath}\n"
        "Run the GUI curation and click Save first."
    )

# --- choose an output path that doesn't already exist ---
out = base_out
if out.exists():
    stem = out.name.replace(".zarr", "")
    parent = out.parent
    k = 2
    while True:
        candidate = parent / f"{stem}-v{k}.zarr"
        if not candidate.exists():
            out = candidate
            break
        k += 1

print(f"Wrote: {out}")

# --- load curation dict ---
with open(curation_filepath, "r") as f:
    curation_dict = json.load(f)

# --- apply curation and save ---
clean_analyzer = apply_curation(sorting_analyzer, curation_dict_or_model=curation_dict)
clean_analyzer = clean_analyzer.save_as(format="zarr", folder=out)


Wrote: /home/marco/codespace/data/raw_intan/Session1/sorted/clean_analyzer.zarr


ValueError: Folder already exists /home/marco/codespace/data/raw_intan/Session1/sorted/clean_analyzer.zarr

In [ ]:
# access the sorting object wrapped by the analyzer, which contains newly updated data

analyzer_obj = load_sorting_analyzer("/home/marco/codespace/data/raw_intan/Session1/sorted/clean_analyzer.zarr")

sorting_obj = analyzer_obj.sorting

In [35]:
# Check out the contents

unit_ids = sorting_obj.unit_ids
sampling_frequency = sorting_obj.sampling_frequency
print(f"Unit IDs: {unit_ids}")
print(f"Sampling Frequency: {sampling_frequency} Hz")

Unit IDs: [0 1 2]
Sampling Frequency: 20000.0 Hz


In [ ]:
import spikeinterface.widgets as sw

fig_dir = base_folder / "figs"
fig_dir.mkdir(parents = True, exist_ok = True)

w = sw.plot_unit_summary(
    analyzer_obj,
    unit_id=id,
    backend="matplotlib",
    ncols=5,
    figsize=(16, 9),
    figtitle=f"Unit summary: {id}"
    )

w.figure.savefig(
    Path(fig_dir, f"{id}-figure")
)

w

In [36]:
# Check out the spike times for some unit

unit_to_get = unit_ids[1] # Get the first unit
spike_train_indices = sorting_obj.get_unit_spike_train(unit_id=unit_to_get, segment_index=0) # segment_index=0 for single-segment data

spike_times = sorting_obj.get_unit_spike_train(unit_id=unit_to_get)

sampling_frequency = sorting_obj.get_sampling_frequency()

spike_times_sec = spike_times / sampling_frequency

print(spike_times_sec[:100])



[  67.39815   86.59875  101.72435  309.9909   362.3398   448.71195
  448.71295  448.7136   448.71535  448.71815  448.72015  448.7218
  448.7238   448.7247   448.72635  448.72735  448.72935  448.73
  448.73075  448.73165  449.37555  449.38275  449.3859   449.7113
  449.78585  449.8102   449.82305  449.84995  449.8594   449.86345
  449.8917   449.924    449.93585  449.9688   450.22705  450.2944
  450.4288   450.49085  450.55395  450.6628   452.83035  452.85725
  452.8827   452.8962   452.97115  453.00585  453.00735  453.1401
  453.2472   464.0453   488.5775   488.5784   505.8041   514.67675
  538.40045  538.6569   560.85175  629.6421   641.1497   644.44575
  662.1661   669.51725  716.77635  764.60435  767.243    841.95165
  861.48415  887.92215  896.9521   908.32     916.97915  939.84115
  945.00955  945.03035  945.04185  945.0441   945.0633   945.24005
  945.87905  947.7597   947.7843   947.86495  947.8838   947.93655
  970.11615  978.5235  1031.3383  1110.5216  1192.63925 1192.64815]


In [ ]:
# Interact with boc via api

!box folders:items 352606395707

----- Folder 352606396623 -----
Type: folder
ID: '352606396623'
Sequence ID: '0'
ETag: '0'
Name: Intan_RDH_2000

----- Folder 354522525287 -----
Type: folder
ID: '354522525287'
Sequence ID: '0'
ETag: '0'
Name: Intan_RDH_2000 (1)


In [28]:
!box folders:items 352606396623

----- Folder 352605477299 -----
Type: folder
ID: '352605477299'
Sequence ID: '0'
ETag: '0'
Name: Session1

----- Folder 352604968054 -----
Type: folder
ID: '352604968054'
Sequence ID: '0'
ETag: '0'
Name: Session2


In [29]:
!box folders:items 352605477299

----- Folder 352607353389 -----
Type: folder
ID: '352607353389'
Sequence ID: '0'
ETag: '0'
Name: raw

----- Folder 354623627238 -----
Type: folder
ID: '354623627238'
Sequence ID: '0'
ETag: '0'
Name: sorted


In [30]:
!box folders:upload 354623627238

Could not read directory 354623627238


In [58]:
!box folders:upload "/home/marco/codespace/data/Intan_RDH_2000/Session1/sorted/cleaned_analyzer.zarr" -p 354623627238

Type: folder
ID: '355655922049'
Sequence ID: '0'
ETag: '0'
Name: cleaned_analyzer.zarr
Created At: '2025-12-12T11:40:22-08:00'
Modified At: '2025-12-12T11:40:22-08:00'
Description: ''
Size: 0
Path Collection:
    Total Count: 5
    Entries:
        -
            Type: folder
            ID: '0'
            Sequence ID: null
            ETag: null
            Name: All Files
        -
            Type: folder
            ID: '352606395707'
            Sequence ID: '0'
            ETag: '0'
            Name: Cloud_Sorter
        -
            Type: folder
            ID: '352606396623'
            Sequence ID: '0'
            ETag: '0'
            Name: Intan_RDH_2000
        -
            Type: folder
            ID: '352605477299'
            Sequence ID: '0'
            ETag: '0'
            Name: Session1
        -
            Type: folder
            ID: '354623627238'
            Sequence ID: '0'
            ETag: '0'
            Name: sorted
Created By:
    Type: user
    ID: '445